In [1]:
import torch
from joblib import dump, load
import torch.utils.data as Data
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import Transformer, TransformerEncoder, TransformerEncoderLayer

torch.manual_seed(100)  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def dataloader(batch_size, workers=2):
    
    train_set = load('train_xdata')
    train_label = load('train_ylabel')
    
    test_set = load('test_xdata')
    test_label = load('test_ylabel')

    train_loader = Data.DataLoader(dataset=Data.TensorDataset(train_set, train_label),
                                   batch_size=batch_size, num_workers=workers, drop_last=True)
    test_loader = Data.DataLoader(dataset=Data.TensorDataset(test_set, test_label),
                                  batch_size=batch_size, num_workers=workers, drop_last=False)
    return train_loader, test_loader

batch_size = 128

train_loader, test_loader = dataloader(batch_size)

In [ ]:
print(len(train_loader))
print(len(test_loader))

In [19]:
# Define the Transformer Model model
class TransformerModel(nn.Module):
    def __init__(self, batch_size, input_dim, unsampling_dim, hidden_dim, num_layers, num_heads, output_dim, output_size, dropout_rate = 0.5):
    
        super().__init__()
        self.batch_size = batch_size

        self.unsampling = nn.Conv1d(input_dim, unsampling_dim, 1)

         # Transformer layers
        self.transformer = TransformerEncoder(
            TransformerEncoderLayer(unsampling_dim, num_heads, hidden_dim, dropout=dropout_rate, batch_first=True),
            num_layers
        )
        
        self.avgpool = nn.AdaptiveAvgPool1d(1)

        self.linear  = nn.Linear(unsampling_dim, output_dim * output_size)
       
    def forward(self, input_seq): 
        unsampling = self.unsampling(input_seq.permute(0,2,1))  

        transformer_output = self.transformer(unsampling.permute(0,2,1)) 
        
        x = self.avgpool(transformer_output.permute(0, 2, 1))          
        
        flat_tensor = x.view(x.size(0), -1) 
        predict = self.linear(flat_tensor)    
        return predict

In [ ]:
batch_size = 128
input_dim = 23  

hidden_dim = 128  
num_layers = 4   
num_heads = 2    

unsampling_dim = 16 
output_dim = 1 
output_size = 7 

model = TransformerModel(batch_size, input_dim, unsampling_dim, hidden_dim, num_layers, num_heads, output_dim, output_size)  

# Define loss function and optimization function
loss_function = nn.MSELoss(reduction='sum')  # loss
learn_rate = 0.0001
optimizer = torch.optim.Adam(model.parameters(), learn_rate)  

def count_parameters(model):
    params = [p.numel() for p in model.parameters() if p.requires_grad]
    for item in params:
        print(f'{item:>6}')
    print(f'______\n{sum(params):>6}')

count_parameters(model)

In [ ]:
print(model)

In [ ]:
import time
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family='Microsoft YaHei')

def model_train(batch_size, epochs, model, optimizer, loss_function, train_loader, test_loader):
    model = model.to(device)
   
    train_size = len(train_loader) * batch_size
    test_size = len(test_loader) * batch_size
 
    minimum_mse = 1000.
    best_model = model

    train_mse = []     
    test_mse = []      

    start_time = time.time()
    for epoch in range(epochs):
         
        model.train()

        train_mse_loss = 0.    
        for seq, labels in train_loader: 
            seq, labels = seq.to(device), labels.to(device)
            
            optimizer.zero_grad()
            y_pred = model(seq)  
            loss = loss_function(y_pred, labels)
            train_mse_loss += loss.item() 
            # Backpropagation and parameter updates
            loss.backward()
            optimizer.step()
        #     break
        # break
        # Calculate the total loss
        train_av_mseloss = train_mse_loss/train_size
        train_mse.append(train_av_mseloss)

        print(f'Epoch: {epoch+1:2} train_MSE-Loss: {train_av_mseloss:10.8f}')
       
        with torch.no_grad():
            test_mse_loss = 0.    
            for data, label in test_loader:
                data, label = data.to(device), label.to(device)
                pre = model(data)
                
                test_loss = loss_function(pre, label)
                test_mse_loss += test_loss.item()
            
            test_av_mseloss = test_mse_loss/test_size
            test_mse.append(test_av_mseloss)
            print(f'Epoch: {epoch+1:2} test_MSE_Loss:{test_av_mseloss:10.8f}')
          
            if test_av_mseloss < minimum_mse:
                minimum_mse = test_av_mseloss
                best_model = model
    

    torch.save(best_model, 'best_model_transformer.pt')
    print(f'\nDuration: {time.time() - start_time:.0f} seconds')
    
    # Visualization
    plt.plot(range(epochs), train_mse, color = 'b',label = 'train_MSE-loss')
    plt.plot(range(epochs), test_mse, color = 'y',label = 'test_MSE-loss')
    plt.legend()
    plt.show()   
    print(f'min_MSE: {minimum_mse}')


epochs = 200
model_train(batch_size, epochs, model, optimizer, loss_function, train_loader, test_loader)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load('best_model_transformer.pt')
model = model.to(device)

train_original_data = []
train_pre_data = []
with torch.no_grad():    
        for data, label in train_loader:      
            origin_lable = label.tolist()    
            train_original_data += origin_lable    

            model.eval()  
            data, label = data.to(device), label.to(device)
            
            train_pred = model(data)  
            train_pred = train_pred.tolist() 
            train_pre_data += train_pred

In [22]:
train_original_data = np.array(train_original_data)
train_pre_data = np.array(train_pre_data)
print('Data Shape：')
print(train_original_data.shape, train_pre_data.shape)

scaler  = load('scaler ')
train_original_data = scaler.inverse_transform(train_original_data)
train_pre_data = scaler.inverse_transform(train_pre_data)

Data Shape：
(13568, 7) (13568, 7)


In [ ]:
import numpy as np
import spotpy.objectivefunctions as evaluation

for step in range(train_original_data.shape[1]):
    print('*' * 50)
    print(f'Evaluate the T+{step+1} prediction on the training dataset')
    
    # NSE
    train_nse = round(evaluation.nashsutcliffe(train_original_data[:, step], train_pre_data[:, step]), 2)
    print(f'NSE (T + {step + 1}):', train_nse)
    
    # VE
    ve_numerator = np.sum(np.abs(train_original_data[:, step] - train_pre_data[:, step]))
    ve_denominator = np.sum(train_original_data[:, step])
    ve = 1 - (ve_numerator / ve_denominator)
    ve_percentage = ve * 100
    train_ve = round(ve_percentage, 1)
    print(f'VE (T + {step + 1}):', train_ve)
    
    # RMSE
    train_rmse = round(evaluation.rmse(train_original_data[:, step], train_pre_data[:, step]), 1)
    print(f'RMSE (T + {step + 1}):', train_rmse)
    
    # MAE
    train_mae = round(evaluation.mae(train_original_data[:, step], train_pre_data[:, step]), 1)
    print(f'MAE (T + {step + 1}):', train_mae)

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = torch.load('best_model_transformer.pt')
model = model.to(device)

test_original_features = []

original_data = []
pre_data = []
with torch.no_grad():    
        for data, label in test_loader:      
            test_original_features += data.tolist()  
            model.eval()  
            origin_lable = label.tolist()    
            original_data += origin_lable    
            
            data, label = data.to(device), label.to(device)
            
            test_pred = model(data)  
            test_pred = test_pred.tolist() 
            pre_data += test_pred

In [ ]:
original_data = np.array(original_data)
pre_data = np.array(pre_data)
print('Data Shape：')
print(original_data.shape, pre_data.shape)

scaler  = load('scaler ')
original_data = scaler.inverse_transform(original_data)
pre_data = scaler.inverse_transform(pre_data)

In [ ]:
import numpy as np
import spotpy.objectivefunctions as evaluation
import pandas as pd

for step in range(original_data.shape[1]):
    print('*' * 50)
    print(f'Evaluate the T+{step+1} prediction on the test dataset')
    
    # NSE
    test_nse = round(evaluation.nashsutcliffe(original_data[:, step], pre_data[:, step]), 2)
    print(f'NSE (T + {step + 1}):', test_nse)
    
    # VE
    ve_numerator = np.sum(np.abs(original_data[:, step] - pre_data[:, step]))
    ve_denominator = np.sum(original_data[:, step])
    ve = 1 - (ve_numerator / ve_denominator)
    ve_percentage = ve * 100
    test_ve = round(ve_percentage, 1)
    print(f'VE (T + {step + 1}):', test_ve)
    
    # RMSE
    test_rmse = round(evaluation.rmse(original_data[:, step], pre_data[:, step]), 1)
    print(f'RMSE (T + {step + 1}):', test_rmse)
    
    # MAE
    test_mae = round(evaluation.mae(original_data[:, step], pre_data[:, step]), 1)
    print(f'MAE (T + {step + 1}):', test_mae)

    df = pd.DataFrame({
        f'True{step+1}': original_data[:, step],
        f'Pred{step+1}': pre_data[:, step]
    })
        
    df.to_csv(f'Transformer_test (T + {step + 1}).csv', index=False, encoding='utf-8')

In [ ]:
batch_size = 128  

model = torch.load('best_model_transformer.pt')
model = model.to(device)
scaler  = load('scaler ')

def model_prediction(model, data, batch_size, scaler):
    temp_data = data.repeat(batch_size, 1, 1)  
    my_pre = model(temp_data)
    my_pre = np.array(my_pre.tolist())
   
    my_pre = scaler.inverse_transform(my_pre)

    pre = my_pre[0]
    return pre

testset = load('test_xdata') 
my_data = testset[-1]   

pre = model_prediction(model, my_data, batch_size, scaler)
print(pre)

In [ ]:
from turtle import color
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc("font", family='Microsoft YaHei')

forecast_step = 7

labels = []  
for i in range(forecast_step):
    label = f"T+{i+1} step prediction value"
    labels.append(label)

fig, axes = plt.subplots(nrows=forecast_step, ncols=1, figsize=(20, 35), dpi=300)

for i, ax in enumerate(axes):
    ax.plot(original_data[:, i],  color = 'orange',label = '真实值')
    ax.plot(pre_data[:, i] , color = 'green', label = labels[i]) 
    ax.set_title(f'Transformer {labels[i]}')
    ax.legend(fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
tensor_data = torch.tensor(test_original_features) 
tensor_data.size()

In [24]:
import shap
tensor_data = tensor_data.to(device)
explainer = shap.DeepExplainer(model, tensor_data)

model.train()
shap_values = explainer.shap_values(tensor_data, check_additivity=False)  

model.eval()
data_numpy = tensor_data.cpu().detach().numpy()  

unrecognized nn.Module: LayerNorm


In [ ]:
print(type(shap_values))         
print(len(shap_values))          
print(type(shap_values[0]))      
print(shap_values[0].shape)      

In [ ]:
shap_values_first_output = shap_values[0]
print(shap_values_first_output.shape)

In [ ]:
shap_values_first_output.shape, data_numpy.shape

In [ ]:
shap.summary_plot(shap_values_first_output[:, 0, :], data_numpy[:, 0, :])

In [ ]:
shap.summary_plot(shap_values_first_output[:, 0, :], data_numpy[:, 0, :], 
                  plot_type="bar")